# SR3 Reader

* Reads data from SR3 file (HDF).
* Return groups, wells, special and grid properties list.
* Return 2D and 3D Time vector.
* Return a 2D or 3D property.
* Save all data to a csv file.

In [2]:
import h5py
from datetime import datetime, timedelta
import re
import numpy as np

In [3]:
with h5py.File(r'..\sr3\base_case_3a.sr3', 'r') as file:
    print("Main Groups in the HDF file:")
    for dataset in file.keys():
        print(f'  {dataset}')

    print("\nAll Groups in the HDF file:")
    def get_type(name):
        print(f'   {name:}\t{type(file[name])}')
    file.visit(get_type)

Main Groups in the HDF file:
  General
  SpatialProperties
  Tables
  TimeSeries

All Groups in the HDF file:
   General	<class 'h5py._hl.group.Group'>
   General/ComponentTable	<class 'h5py._hl.dataset.Dataset'>
   General/EventTable	<class 'h5py._hl.dataset.Dataset'>
   General/HistoryTable	<class 'h5py._hl.dataset.Dataset'>
   General/MasterTimeTable	<class 'h5py._hl.dataset.Dataset'>
   General/NameRecordTable	<class 'h5py._hl.dataset.Dataset'>
   General/TableAssociations	<class 'h5py._hl.dataset.Dataset'>
   General/UnitConversionTable	<class 'h5py._hl.dataset.Dataset'>
   General/UnitsTable	<class 'h5py._hl.dataset.Dataset'>
   SpatialProperties	<class 'h5py._hl.group.Group'>
   SpatialProperties/000000	<class 'h5py._hl.group.Group'>
   SpatialProperties/000000/BKRGCL	<class 'h5py._hl.dataset.Dataset'>
   SpatialProperties/000000/BKRGRL	<class 'h5py._hl.dataset.Dataset'>
   SpatialProperties/000000/BKROCRW	<class 'h5py._hl.dataset.Dataset'>
   SpatialProperties/000000/BKROCW	<cl

In [4]:
f = h5py.File(r'..\sr3\base_case_3a.sr3', 'r')

## Master Data

In [5]:
dataset = f['General/MasterTimeTable']
day_list = {number:days for (number,days) in zip(dataset['Index'], dataset['Offset in days'])}

def parse_date(date):
    date_string = str(date)
    integer_part = int(date_string.split('.')[0])
    decimal_part = float("0." + date_string.split('.')[1])

    parsed_date = datetime.strptime(str(integer_part), "%Y%m%d")
    fraction_of_day = timedelta(days=decimal_part)
    return parsed_date + fraction_of_day

dataset = f['General/MasterTimeTable']
date_list = {number:parse_date(date) for (number,date) in zip(dataset['Index'], dataset['Date'])}

In [6]:
dataset = f['General/UnitsTable']
unit_list = {number:{'internal':in_name.decode(), 'current':out_name.decode(), 'conversion':dict()} for (number,in_name,out_name) in zip(dataset['Index'],dataset['Internal Unit'],dataset['Output Unit'])}

dataset = f['General/UnitConversionTable']
for (number, name, gain, offset) in zip(dataset['Dimensionality'],dataset['Unit Name'],dataset['Gain'],dataset['Offset']):
    unit_list[number]['conversion'][name.decode()] = (1./gain, offset * (-1.))

for d in unit_list.values():
    if d['internal'] != d['current']:
        gain, offset = d['conversion'][d['internal']]
        for k in d['conversion']:
            g, o = d['conversion'][k]
            d['conversion'][k] = (g / gain, o - offset)

def set_current_unit(dimensionality, unit):
    if unit not in unit_list[dimensionality]['conversion']:
        raise ValueError(f'{unit} is not a valid unit for the current dimension.')
    unit_list[dimensionality]['current'] = unit

def unit_conversion(value, dimensionality, unit=None):
    if unit is None:
        unit = unit_list[dimensionality]['current']
    if unit == unit_list[dimensionality]['internal']:
        return value
    gain, offset = unit_list[dimensionality]['conversion'][unit]
    if gain == 1. and offset == 0.:
        return value
    else:
        return value * gain + offset

In [7]:
dataset = f['General/ComponentTable']
component_list = {(number+1):name[0].decode() for (number,name) in enumerate(dataset[:])}

def replace_components_property_list(property_list):
    pattern = re.compile(r'\((\d+)\)')
    def replace(match):
        number = int(match.group(1))
        return f'({str(component_list.get(number, match.group(1)))})'
    return {pattern.sub(replace, k):v for k,v in property_list.items()}

In [8]:
dataset = f['General/NameRecordTable']
property_list = dict()
for (keyword,name,long_name,dimensionality) in zip(dataset['Keyword'],dataset['Name'],dataset['Long Name'],dataset['Dimensionality']):
    if keyword != '':
        keyword = keyword.decode()
        name = name.decode()
        long_name = long_name.decode()
        dimensionality = dimensionality.decode()
        if keyword[-2:] == '$C':
            for c in component_list.values():
                property_list[f'{keyword[:-2]}({c})'] = {'name':name.replace('$C', f' ({c})'), 'long name':long_name.replace('$C', f' ({c})'), 'dimensionality_string':dimensionality}
        else:
            property_list[keyword] = {'name':name, 'long name':long_name, 'dimensionality_string':dimensionality}


In [9]:
def unit_from_dimensionality(dimensionality_string):
    if dimensionality_string == '':
        return ''
    unit = ''
    if dimensionality_string[0] == '-':
        unit = '1'
    d = ''
    for c in dimensionality_string:
        if c == '|':
            unit = unit + unit_list[int(d)]['current']
            d = ''
        elif c == '-':
            unit = unit + '/'
        else:
            d = d + c
    return unit

def unit_from_property(property_):
    return unit_from_dimensionality(property_list[property_]['dimensionality_string'])

### Property Formulas

* Basic
    * 1035: sum (2 parameters)
    * 1037: sum (3 parameters)

* Time derivatives
    * 1040: derivative (1 parameter)
    * 1045: monthly derivative (1 parameter)
    * 1046: quarterly derivative (1 parameter)
    * 1047: yearly derivative (1 parameter)
    * 1048: daily derivative (1 parameter)
    * 1049: weekly derivative (1 parameter)

* Percentage
    * 1055: A*100 (1 parameter)

* Division
    * 1060: division (2 parameters)
    * 1062: division * 100 (2 parameters) <= used with on-fraction

    * 1080: division (2 parameters)

* Previous month
    * 1121: monthly derivative of previous month (1 parameter)
    * 1122: yearly derivative of previous month (1 parameter)

* Grid properties(?)
    * 1130: A*grid height (1 parameter)
    * 1140: A/(B*C) ? (3 parameters)
    * 1160: A*B (2 parameters)
    * 1200: ternary (2 parameters)
    * 1210: sum over layers (1 parameter)

* Per sector
    * 1110: ??? (1 parameter) <= per sector
    * 1245: ??? (1 parameter) <= per sector
    * 1246: derivative of 1245 (1 parameter) 



## Time Series

In [89]:
_element = {'special':{'':0}}
_parent = dict()
_connection = dict()
_property = dict()
_timestep = dict()
_day = dict()
_date = dict()
# _data = dict()

def _return_dataset(element_type, dataset_string):
    el_type_string = element_type.upper() + 'S'
    if element_type == 'special':
        el_type_string = el_type_string + ' HISTORY'
    s = f'TimeSeries/{el_type_string}/{dataset_string}'
    if s in f:
        return f[s]
    else:
        raise ValueError(f'Dataset {dataset_string} not found for {element_type}.')
    
def get_elements(element_type):        
    if element_type not in _element:
        dataset = _return_dataset(element_type=element_type, dataset_string='Origins')
        _element[element_type] = {name.decode():number for (number,name) in enumerate(dataset[:]) if name.decode()!=''}
    return _element[element_type]

def get_properties(element_type):
    if element_type not in _property:
        dataset = _return_dataset(element_type=element_type, dataset_string='Variables')
        _property[element_type] = {name.decode():number for (number,name) in enumerate(dataset[:])}
        _property[element_type] = replace_components_property_list(_property[element_type])
    return _property[element_type]

def get_timesteps(element_type):
    if element_type not in _timestep:
        dataset = _return_dataset(element_type=element_type, dataset_string='Timesteps')
        _timestep[element_type] = dataset[:]
    return _timestep[element_type]

def get_days(element_type):
    if element_type not in _day:
        timesteps = get_timesteps(element_type=element_type)
        _day[element_type] = np.vectorize(lambda x: day_list[x])(timesteps)
    return _day[element_type]

def get_dates(element_type):
    if element_type not in _date:
        timesteps = get_timesteps(element_type=element_type)
        _date[element_type] = np.vectorize(lambda x: date_list[x])(timesteps)
    return _date[element_type]

def _get_parents(element_type):
    if element_type in ['well', 'group', 'layer']:
        dataset = _return_dataset(element_type=element_type, dataset_string=f'{element_type.capitalize()}Table')
        def _name(name, parent):
            if element_type == 'layer':
                return f'{parent.decode()}{{{name.decode()}}}'
            else:
                return name.decode()
        _parent[element_type] = {_name(name, parent):parent.decode() for (name,parent) in zip(dataset['Name'], dataset['Parent'])}
    else:
        _parent[element_type] = {name:'' for name in get_elements(element_type)}

def get_parent(element_type, element_name):
    if element_type not in _parent:
        _get_parents(element_type)
    return _parent[element_type][element_name]

def _get_connections(element_type):
    if element_type == 'layer':
        dataset = _return_dataset(element_type=element_type, dataset_string=f'{element_type.capitalize()}Table')
        def _name(name, parent):
            return f'{parent.decode()}{{{name.decode()}}}'
        _connection[element_type] = {_name(name, parent):connection for (name,parent,connection) in zip(dataset['Name'], dataset['Parent'], dataset['Connect To'])}
    else:
        _parent[element_type] = {name:'' for name in get_elements(element_type)}

def get_connection(element_type, element_name):
    if element_type not in _connection:
        _get_connections(element_type)
    return _connection[element_type][element_name]

def _concatenate_arrays(arr1, arr2):
    # common_dtype = np.result_type(arr1, arr2)
    # arr1 = arr1.astype(common_dtype)
    # arr2 = arr2.astype(common_dtype)

    if arr1.ndim == 1:
        arr1 = arr1.reshape(-1,1)
    if arr2.ndim == 1:
        arr2 = arr2.reshape(-1,1)

    return np.hstack((arr1, arr2))

def get_data(element_type, property_name, element_name='', with_dates=False, with_days=False, with_timesteps=False):
    dataset = _return_dataset(element_type=element_type, dataset_string='Data')
    data = dataset[:,get_properties(element_type)[property_name],get_elements(element_type)[element_name]]
    if with_dates:
        return _concatenate_arrays(get_dates(element_type=element_type), data)
    elif with_days:
        return _concatenate_arrays(get_days(element_type=element_type), data)
    elif with_timesteps:
        return _concatenate_arrays(get_timesteps(element_type=element_type), data)
    else:
        return data


In [11]:
print(get_elements('well')['P11'])
print(get_elements('well')['P12'])

print(get_properties('well')['OILRATSC'])
print(get_properties('well')['GASRATSC'])


0
2
20
21


In [12]:
dataset = _return_dataset(element_type='well', dataset_string='Data')

In [13]:
type(dataset[:,[20,21],0])

numpy.ndarray

In [88]:
z=concatenate_data(x2,a)
z[:]

array([[datetime.datetime(2018, 10, 2, 0, 0), 6246.5344882818345, 0.0],
       [datetime.datetime(2018, 10, 31, 23, 45, 36), 6246.517578890559,
        0.0],
       [datetime.datetime(2018, 11, 1, 0, 0), 6394.106652683749, 0.0],
       ...,
       [datetime.datetime(2024, 8, 1, 23, 45, 36), 5083.7696492729665,
        4759.4954772827605],
       [datetime.datetime(2024, 8, 2, 0, 0), 6082.504288854868, 0.0],
       [datetime.datetime(2024, 8, 3, 0, 0), 6078.931879073354, 0.0]],
      dtype=object)

In [73]:
a.ndim

2

In [74]:
x.ndim

1

In [61]:
a = dataset[:,20,[0,2]]
a.shape

(658, 2)

In [64]:
concatenate_data(x2,a)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 658 and the array at index 1 has size 1316

In [63]:
a

array([[6246.53448828,    0.        ],
       [6246.51757889,    0.        ],
       [6394.10665268,    0.        ],
       ...,
       [5083.76964927, 4759.49547728],
       [6082.50428885,    0.        ],
       [6078.93187907,    0.        ]])

In [52]:
x = get_days('well')
x2 = get_dates('well')
y = get_data('well','GASRATSC','P11')

In [35]:
print(type(x))
print(x.shape)
print(len(x))

print(type(y))
print(y.shape)
print(len(y))

<class 'numpy.ndarray'>
(658,)
658
<class 'numpy.ndarray'>
(658,)
658


In [51]:
z = concatenate_data(x,y)
z[0,1]
# z[:,0]

2691745.811279528

In [60]:
z2 = concatenate_data(x2,y)
type(z2)

numpy.ndarray

In [49]:
y[0]

2691745.811279528

In [41]:
z = np.hstack((x.reshape(-1,1),y.reshape(-1,1)))
z[0]

array([3.00000000e+01, 2.69174581e+06])

In [42]:
z

array([[3.00000000e+01, 2.69174581e+06],
       [5.99900000e+01, 2.69173852e+06],
       [6.00000000e+01, 2.75533735e+06],
       ...,
       [2.16099000e+03, 2.19225966e+06],
       [2.16100000e+03, 2.62282463e+06],
       [2.16200000e+03, 2.62131320e+06]])

In [20]:
z = np.vstack((x,y))
print(z.shape)

(2, 658)


In [21]:
z

array([[datetime.datetime(2018, 10, 2, 0, 0),
        datetime.datetime(2018, 10, 31, 23, 45, 36),
        datetime.datetime(2018, 11, 1, 0, 0), ...,
        datetime.datetime(2024, 8, 1, 23, 45, 36),
        datetime.datetime(2024, 8, 2, 0, 0),
        datetime.datetime(2024, 8, 3, 0, 0)],
       [2691745.811279528, 2691738.5248579876, 2755337.352646691, ...,
        2192259.657024043, 2622824.6299249204, 2621313.202984272]],
      dtype=object)

In [91]:
get_data('well','OILRATSC','P11', with_days=True)

array([[  30.        , 6246.53448828],
       [  59.99      , 6246.51757889],
       [  60.        , 6394.10665268],
       ...,
       [2160.99      , 5083.76964927],
       [2161.        , 6082.50428885],
       [2162.        , 6078.93187907]])

In [ ]:
dataset = f['TimeSeries/WELLS/Origins']
_element['well'] = {name.decode():number for (number,name) in enumerate(dataset[:]) if name.decode()!=''}

dataset = f['TimeSeries/WELLS/WellTable']
_parent['well'] = {name.decode():parent.decode() for (name,parent) in zip(dataset['Name'], dataset['Parent'])}

dataset = f['TimeSeries/WELLS/Variables']
_property['well'] = {name.decode():number for (number,name) in enumerate(dataset[:])}
_property['well'] = replace_components_property_list(_property['well'])

dataset = f['TimeSeries/WELLS/Timesteps']
_timestep['well'] = {number:timestep for (number,timestep) in enumerate(dataset[:])}

_day['well'] = [day_list[ts] for ts in _timestep['well'].values()]
_date['well'] = [date_list[ts] for ts in _timestep['well'].values()]


In [ ]:
dataset = f['TimeSeries/GROUPS/Origins']
group_list = {name.decode():number for (number,name) in enumerate(dataset[:]) if name.decode()!=''}

dataset = f['TimeSeries/GROUPS/GroupTable']
group_parent = {name.decode():parent.decode() for (name,parent) in zip(dataset['Name'], dataset['Parent'])}

dataset = f['TimeSeries/GROUPS/Variables']
group_property_list = {name.decode():number for (number,name) in enumerate(dataset[:])}
group_property_list = replace_components_property_list(group_property_list)

dataset = f['TimeSeries/GROUPS/Timesteps']
group_timestep_list = {number:timestep for (number,timestep) in enumerate(dataset[:])}

group_day_list = [day_list[ts] for ts in group_timestep_list.values()]
group_date_list = [date_list[ts] for ts in group_timestep_list.values()]

def get_group_property(property_name, group_name):
    dataset = f['TimeSeries/GROUPS/Data']
    return dataset[:,group_property_list[property_name],group_list[group_name]]

In [ ]:
dataset = f['TimeSeries/LAYERS/Origins']
layer_list = {name.decode():number for (number,name) in enumerate(dataset[:]) if name.decode()!=''}

dataset = f['TimeSeries/LAYERS/LayerTable']
layer_parent = {f'{parent.decode()}{{{name.decode()}}}':parent.decode() for (name,parent) in zip(dataset['Name'], dataset['Parent'])}

dataset = f['TimeSeries/LAYERS/LayerTable']
layer_connection = {f'{parent.decode()}{{{name.decode()}}}':connection for (name,parent,connection) in zip(dataset['Name'], dataset['Parent'], dataset['Connect To'])}

dataset = f['TimeSeries/LAYERS/Variables']
layer_property_list = {name.decode():number for (number,name) in enumerate(dataset[:])}
layer_property_list = replace_components_property_list(layer_property_list)

dataset = f['TimeSeries/LAYERS/Timesteps']
layer_timestep_list = {number:timestep for (number,timestep) in enumerate(dataset[:])}

layer_day_list = [day_list[ts] for ts in layer_timestep_list.values()]
layer_date_list = [date_list[ts] for ts in layer_timestep_list.values()]

def get_layer_property(property_name, layer_name):
    dataset = f['TimeSeries/LAYERS/Data']
    return dataset[:,layer_property_list[property_name],layer_list[layer_name]]

In [ ]:
dataset = f['TimeSeries/SECTORS/Origins']
sector_list = {name.decode():number for (number,name) in enumerate(dataset[:]) if name.decode()!=''}

dataset = f['TimeSeries/SECTORS/Variables']
sector_property_list = {name.decode():number for (number,name) in enumerate(dataset[:])}
sector_property_list = replace_components_property_list(sector_property_list)

dataset = f['TimeSeries/SECTORS/Timesteps']
sector_timestep_list = {number:timestep for (number,timestep) in enumerate(dataset[:])}

sector_day_list = [day_list[ts] for ts in sector_timestep_list.values()]
sector_date_list = [date_list[ts] for ts in sector_timestep_list.values()]

def get_sector_property(property_name, sector_name):
    dataset = f['TimeSeries/SECTORS/Data']
    return dataset[:,sector_property_list[property_name],sector_list[sector_name]]

In [ ]:
dataset = f['TimeSeries/SPECIAL HISTORY/Variables']
special_property_list = {name.decode():number for (number,name) in enumerate(dataset[:])}

dataset = f['TimeSeries/SPECIAL HISTORY/Timesteps']
special_timestep_list = {number:timestep for (number,timestep) in enumerate(dataset[:])}
special_property_list = replace_components_property_list(special_property_list)

special_day_list = [day_list[ts] for ts in special_timestep_list.values()]
special_date_list = [date_list[ts] for ts in special_timestep_list.values()]

def get_special_property(property_name):
    dataset = f['TimeSeries/SPECIAL HISTORY/Data']
    return dataset[:,special_property_list[property_name],0]

## Grid Properties

In [ ]:
dataset = f['SpatialProperties']
grid_timestep_list = dict()
i = 0
for key in dataset.keys():
    sub_dataset = f[f"SpatialProperties/{key}"]
    if isinstance(sub_dataset, h5py._hl.group.Group):
        grid_timestep_list[i] = key
        i += 1
    
grid_day_list = [day_list[int(ts)] for ts in grid_timestep_list.values()]
grid_date_list = [date_list[int(ts)] for ts in grid_timestep_list.values()]

In [ ]:
dataset = f['SpatialProperties/000000/GRID']
ni = dataset['IGNTID'][0]
nj = dataset['IGNTJD'][0]
nk = dataset['IGNTKD'][0]

n_cells = ni*nj*nk
print(f'{ni}x{nj}x{nk} = {n_cells} cells')

47x39x291 = 533403 cells


In [ ]:
dataset = f['SpatialProperties/Statistics']
grid_property_list = {name.decode().replace('/','%2F'):{'min':min_, 'max':max_, 'timesteps':set()} for name,min_,max_ in zip(dataset['Keyword'],dataset['Min'],dataset['Max'])}

def _list_grid_properties(timestep, add_timestep=True):
    dataset = f[f'SpatialProperties/{timestep}']
    for key in dataset.keys():
        sub_dataset = f[f"SpatialProperties/{timestep}/{key}"]
        if isinstance(sub_dataset, h5py._hl.dataset.Dataset):
            if key in grid_property_list:
                if 'size' in grid_property_list[key]:
                    if grid_property_list[key]['size'] != sub_dataset.size:
                        raise ValueError(f'Inconsistent grid size for {key}.')
                else:
                    grid_property_list[key]['size'] = sub_dataset.size
                if add_timestep:
                    grid_property_list[key]['timesteps'].add(timestep)
            else:
                raise ValueError(f'{key} not listed previously!')

_list_grid_properties('000000/GRID', False)
for ts in grid_timestep_list.values():
    _list_grid_properties(ts)

In [ ]:
n_active_cells = grid_property_list['IPSTCS']['size']
print(f'Number of active cells: {n_active_cells}')

Number of active cells: 67241


In [ ]:
for i in [3,4,234,434,1023,3400,5000]:
    print(f['SpatialProperties/000000/GRID/NODES'][i])

786744.8332000001
7281231.6992999995
776189.1926
5259.2539
785783.4680000001


7277323.4571
5352.8584


In [ ]:
grid_property_list

{'ICSTBC': {'min': -2.0, 'max': 0.0, 'timesteps': set(), 'size': 533403},
 'ICSTCG': {'min': 0.0, 'max': 0.0, 'timesteps': set(), 'size': 533403},
 'ICSTGN': {'min': 1.0, 'max': 1.0, 'timesteps': set(), 'size': 533403},
 'ICSTPS': {'min': 0.0, 'max': 67241.0, 'timesteps': set(), 'size': 533403},
 'ICSTPB': {'min': 1.0, 'max': 533403.0, 'timesteps': set(), 'size': 533403},
 'IPSTCS': {'min': 373.0, 'max': 533075.0, 'timesteps': set(), 'size': 67241},
 'IPSTBT': {'min': 9.0, 'max': 9.0, 'timesteps': set(), 'size': 67241},
 'IPSTAC': {'min': 1.0, 'max': 1.0, 'timesteps': set(), 'size': 67241},
 'IGNTID': {'min': 47.0, 'max': 47.0, 'timesteps': set(), 'size': 1},
 'IGNTJD': {'min': 39.0, 'max': 39.0, 'timesteps': set(), 'size': 1},
 'IGNTKD': {'min': 291.0, 'max': 291.0, 'timesteps': set(), 'size': 1},
 'IGNTZA': {'min': 0.0, 'max': 0.0, 'timesteps': set(), 'size': 1},
 'IGNTGT': {'min': 12.0, 'max': 12.0, 'timesteps': set(), 'size': 1},
 'IGNTFR': {'min': 533404.0, 'max': 533404.0, 'times

In [ ]:
dataset = f['SpatialProperties/000000/GRID']
for key in dataset.keys():
    sub_dataset = f[f"SpatialProperties/000000/GRID/{key}"]
    size = '{:,}'.format(sub_dataset.size)
    print(f'  {key}: {size} -> {property_list[key]["long name"]}')

  BLOCKDEPTH: 533,403 -> Grid block depths for all layers
  BLOCKPVOL: 67,241 -> Block pore volume
  BLOCKS: 4,267,224 -> Grid Block connectivity of Node Based Corner Point Grid
  BLOCKSIZE: 1,600,209 -> Grid block sizes in three directions
  BVOL: 67,241 -> Gross Block Volume = dx*dy*dz
  ICNTDR: 176,572 -> Connection direction
  ICSTBC: 533,403 -> Complete storage to block type
  ICSTCG: 533,403 -> Complete storage to child grid
  ICSTGN: 533,403 -> Complete storage to grid number
  ICSTPB: 533,403 -> Complete storage to parent block
  ICSTPS: 533,403 -> Complete storage to packed storage
  ICTPS1: 176,572 -> Connection to lower packed storage
  ICTPS2: 176,572 -> Connection to upper packed storage
  IGNTFR: 1 -> Grid number to first fracture CS index
  IGNTGT: 1 -> Grid number to grid type
  IGNTID: 1 -> Grid number to no. of I direction blocks
  IGNTJD: 1 -> Grid number to no. of J direction blocks
  IGNTKD: 1 -> Grid number to no. of K direction blocks
  IGNTNC: 2 -> Grid number t

In [ ]:
{p:grid_property_list[p] for p in grid_property_list if len(grid_property_list[p]['timesteps'])==0}

{'ICSTBC': {'min': -2.0, 'max': 0.0, 'timesteps': set(), 'size': 533403},
 'ICSTCG': {'min': 0.0, 'max': 0.0, 'timesteps': set(), 'size': 533403},
 'ICSTGN': {'min': 1.0, 'max': 1.0, 'timesteps': set(), 'size': 533403},
 'ICSTPS': {'min': 0.0, 'max': 67241.0, 'timesteps': set(), 'size': 533403},
 'ICSTPB': {'min': 1.0, 'max': 533403.0, 'timesteps': set(), 'size': 533403},
 'IPSTCS': {'min': 373.0, 'max': 533075.0, 'timesteps': set(), 'size': 67241},
 'IPSTBT': {'min': 9.0, 'max': 9.0, 'timesteps': set(), 'size': 67241},
 'IPSTAC': {'min': 1.0, 'max': 1.0, 'timesteps': set(), 'size': 67241},
 'IGNTID': {'min': 47.0, 'max': 47.0, 'timesteps': set(), 'size': 1},
 'IGNTJD': {'min': 39.0, 'max': 39.0, 'timesteps': set(), 'size': 1},
 'IGNTKD': {'min': 291.0, 'max': 291.0, 'timesteps': set(), 'size': 1},
 'IGNTZA': {'min': 0.0, 'max': 0.0, 'timesteps': set(), 'size': 1},
 'IGNTGT': {'min': 12.0, 'max': 12.0, 'timesteps': set(), 'size': 1},
 'IGNTFR': {'min': 533404.0, 'max': 533404.0, 'times

## Get Coordinates

In [ ]:
_CHUNK_SIZE = 1200

def expand_list(original_list, items=1):
    expanded_list = []
    for value in original_list:
        value_shift = (value-1)*items
        expanded_list.append(value_shift)
        for new_value in range(value_shift + 1, value_shift + items):
            expanded_list.append(new_value)
    return expanded_list

def get_dataset_data(dataset, values_list):
    if not is_iterable(values_list):
        return dataset[values_list]
    x_ordered = list(set(values_list)).copy()
    x_ordered.sort()
    sub_x_ordered = np.array_split(x_ordered, int(len(x_ordered)/_CHUNK_SIZE)+1)
    y_dict = dict()
    for x in sub_x_ordered:
        y = dataset[x]
        for x,y in zip(x, y):
            y_dict[x] = y        
    return  np.array([y_dict[x] for x in values_list])

def is_iterable(obj):
    try:
        iter(obj)
        return True
    except TypeError:
        return False
    
def get_cel_number(i,j,k, can_be_iterable=True):
    if can_be_iterable and is_iterable(i):
        return [ii + ni*(jj-1 + (kk-1)*nj) for (ii,jj,kk) in zip(i,j,k)]
    return i + ni*(j-1 + (k-1)*nj)

def get_nodes_index(i,j=None,k=None, can_be_iterable=True):
    dataset = f['SpatialProperties/000000/GRID/BLOCKS']
    if can_be_iterable and is_iterable(i):
        if j is not None:
            i = get_cel_number(i,j,k)
        y = get_dataset_data(dataset, expand_list(i, 8))
        return np.reshape(y, (-1, 8))      
    if j is not None:
        i = get_cel_number(i,j,k)
    return dataset[(i-1)*8:i*8]

def get_cell_coordinates(i,j=None,k=None, can_be_iterable=True):
    dataset = f['SpatialProperties/000000/GRID/NODES']
    if can_be_iterable and is_iterable(i):
        if j is not None:
            i = get_cel_number(i,j,k)
        nodes = get_nodes_index(i)
        y = get_dataset_data(dataset, expand_list(nodes.flatten(), 3))
        return y.reshape((len(i), 8, 3))
    if j is not None:
        i = get_cel_number(i,j,k)
    return [dataset[(n-1)*3:n*3] for n in get_nodes_index(i)]

In [ ]:
size = 2000
i_list = list(np.random.randint(low=1, high=ni, size=size))
j_list = list(np.random.randint(low=1, high=nj, size=size))
k_list = list(np.random.randint(low=1, high=nk, size=size))

cells = get_cel_number(i_list,j_list,k_list)

start_time = datetime.now()
x = get_cell_coordinates(cells)
end_time = datetime.now()
print(f'Elapsed time: {end_time - start_time}')

Elapsed time: 0:00:04.204510


In [ ]:
def time_function(func, size):
    i_list = list(np.random.randint(low=1, high=ni, size=size))
    j_list = list(np.random.randint(low=1, high=nj, size=size))
    k_list = list(np.random.randint(low=1, high=nk, size=size))
    cells = get_cel_number(i_list,j_list,k_list)

    start_time = datetime.now()
    x = func(cells)
    end_time = datetime.now()
    # print(f'{s} elemts: {end_time - start_time} ({(end_time - start_time)/s*1000} per 1000 elemts)')
    return (end_time - start_time)/size*1000 

def time_function_sens(func, arguments_sizes):
    t = []
    for s in arguments_sizes:
        t.append(time_function(func, s))
    return t

def time_function_opt(func, min_val, max_val, fmin_val=None, fmax_val=None):
    if fmin_val is None:
        fmin_val = time_function(func, min_val)
        print(f'{min_val} => {fmin_val}')
    if fmax_val is None:
        fmax_val = time_function(func, max_val)
        print(f'{max_val} => {fmax_val}')
    a_val = int((2*min_val + max_val)/3)
    b_val = int((min_val + 2*max_val)/3)
    if a_val == min_val or b_val == max_val:
        if fmin_val < fmax_val:
            return min_val
        else:
            return max_val
    fa_val = time_function(func, a_val)
    fb_val = time_function(func, b_val)
    print(f'{a_val} => {fa_val}')
    print(f'{b_val} => {fb_val}')
    if fa_val < fb_val:
        return time_function_opt(func, min_val, b_val, fmin_val, fb_val)
    else:
        return time_function_opt(func, a_val, max_val, fa_val, fmax_val)


# t = time_function_sens(get_nodes_index, [100, 200, 500, 1000, 2000, 5000])
    
